# Marine Biodiversity and Environmental Data Exploration with `mlcroissant`
This notebook provides a step-by-step guide for loading and exploring a dataset using the `mlcroissant` library.

### Dataset Source
The dataset source is provided via a Croissant schema URL.

In [ ]:
# Ensure `mlcroissant` library is installed
!pip install mlcroissant

## 1. Data Loading
Load metadata and records from the dataset using `mlcroissant`.

In [ ]:
import mlcroissant as mlc
import pandas as pd

# Define the dataset URL
url = 'https://cdn.dev.senscience.cloud/portals/10.82843/pm80-mh77/fair2.json'

# Load the dataset metadata
dataset = mlc.Dataset(url)
metadata = dataset.metadata
print(f"{metadata['name']}: {metadata['description']}")

## 2. Data Overview
Review available record sets, fields, and their IDs.

In [ ]:
# List available record sets
record_sets = metadata['recordSet']
for record_set in record_sets:
    print(f"Record Set: {record_set['@id']}, Description: {record_set['description']}")

## 3. Data Extraction
Load data from a specific record set into a DataFrame for analysis. Use the record set and field `@id`s from the overview.

In [ ]:
# Extract data from each record set
record_sets_ids = [
    'https://senscience.ai/frontiers/borja/README.csv',
    'https://senscience.ai/frontiers/borja/METADATA.csv',
    'https://senscience.ai/frontiers/borja/WATER.csv'
]
dataframes = {}

for record_set_id in record_sets_ids:
    records = list(dataset.records(record_set=record_set_id))
    dataframes[record_set_id] = pd.DataFrame(records)

print(dataframes['https://senscience.ai/frontiers/borja/WATER.csv'].columns.tolist())
dataframes['https://senscience.ai/frontiers/borja/WATER.csv'].head()

## 4. Exploratory Data Analysis (EDA)
Apply common data processing steps, such as filtering records based on specific criteria, normalizing numeric fields, and categorizing data. This section should include operations like removing outliers, transforming data distributions, or grouping data by key attributes to prepare it for further analysis.

In [ ]:
# Select a numeric field for analysis
numeric_field = 'maximumdepthinmeters'
record_set_id = 'https://senscience.ai/frontiers/borja/WATER.csv'

threshold = 10
filtered_df = dataframes[record_set_id][dataframes[record_set_id][numeric_field] > threshold]
print(f"Filtered records with {numeric_field} > {threshold}:")
print(filtered_df.head())

filtered_df[f"{numeric_field}_normalized"] = (filtered_df[numeric_field] - filtered_df[numeric_field].mean()) / filtered_df[numeric_field].std()
print(f"Normalized {numeric_field} for filtered records:")
print(filtered_df[[numeric_field, f"{numeric_field}_normalized"]].head())

group_field = 'parameter'
if group_field in dataframes[record_set_id].columns:
    grouped_df = filtered_df.groupby(group_field).mean()
    print(f"Grouped data by {group_field}:")
    print(grouped_df.head())

## 5. Visualization
Visualize data distributions or relationships between fields in the dataset.

In [ ]:
import matplotlib.pyplot as plt

# Plot histogram of parameter values
plt.figure(figsize=(10, 6))
filtered_df[numeric_field].hist(bins=50)
plt.title('Histogram of Maximum Sampling Depth')
plt.xlabel('Depth (meters)')
plt.ylabel('Frequency')
plt.show()

## 6. Conclusion
Summarize key findings and observations from the dataset exploration.

- The dataset provides extensive environmental monitoring data across various aquatic environments.
- Initial data filtering and normalization reveal insights into sampling depths and parameters measured.
- Visual analysis helps in understanding the distribution of key environmental factors.
- Further analysis could involve examining temporal trends or correlations between different environmental variables.